<a href="https://colab.research.google.com/github/AdrianAasen/Bloch_Visualization/blob/main/AQT_Bloch_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Qubit state visualization

AQT 2022

In [62]:
# load standard libraries

import numpy as np   # standard numerics library

import matplotlib.pyplot as plt   # for making plots
!pip install qutip # To install qutip in this session
from qutip import Bloch 


# for interactive graphics
from ipywidgets import interactive, interact
from ipywidgets import FloatSlider, fixed, IntSlider

# test comment

# comment

%matplotlib inline

### Single qubit rotations

We start by implementing common single-qubit rotation gates and visualizing their action on the Bloch sphere.

#### Defining the gates

- Define the Pauli gates, and the Hadamard, S and T gates and the identity gates.
- Define a function that builds a rotation gate rotation(ax, theta) for arbitrary rotation axis ax and angle theta.

In [ ]:
# Define all common single qubit gates
X=np.array([[0,1],[1,0]])
Y=np.array([[0,-1j],[1j,0]])
Z=np.array([[1,0],[0,-1]])
H=1/np.sqrt(2)*np.array([[1,1],[1,-1]])
S=np.array([[1,0],[0,1j]])
T=np.array([[1,0],[0,np.exp(1j*np.pi/4)]])
Id=np.array([[1,0],[0,1]])

# Rotate the Bloch-sphere about axis "ax" by an angle "theta"
def rotation(ax,theta):
  return np.cos(theta/2)*Id -1j*np.sin(theta/2)*(ax[0]*X + ax[1]*Y +ax[2]*Z )


# initially the spin is in the state |0>
ini = np.array([1,0]) # spin up, north pole
axis = np.array([1,0,0]) # rotate about x

# rotate the spin and observe what happens on the Bloch sphere
def f(phi):
  # rotate by an angle phi 
  state = rotation(axis, phi) @ ini
  print(f'Current qubit state {state}')
  # calculate the spin expectation values
  pauliExpVals = state @ [X,Y,Z] @ state.conj().T
  print(f'Pauli expectation values {pauliExpVals}')
  # plot a point on the sphere
  b = Bloch()
  b.add_points(pauliExpVals)
  b.render()


interact(f, phi=FloatSlider(min=0,max=np.pi,step=np.pi/20));

interactive(children=(FloatSlider(value=0.0, description='phi', max=3.141592653589793, step=0.1570796326794896…

### Arbitrary single qubit gate operations

We generate a set of sample states, regularly spaced points on the unit circle in the x-z-plane, and apply some of the gates we defined previously. As an example we have applied some gates, (rotations, H, T, HTH, THTH...), to all of these states. Feel free to make your own 'OpList' and see what the gates does the to states on the Bloch sphere!

In [ ]:
# Define input
nSampels=10 # The points you want in your disk
phi=np.pi/4 # Angle to rotate
rotationAxis=np.array([1,0,0]) # Axis about which you rotate.
# An example of all the operations you can preform on the Bloch sphere, play around!
OpList=np.array([rotation(axis, phi),
                 rotation(axis, 2*phi),
                 H,
                 T,
                 H@T@H,
                 H@T@H@T,
                 T@H@T@H@T])

#List to lable the operations for the slider
OpListText=["Rotation around x-axis by pi/4","Rotation around x-axis by pi/2" ,"H","T","HTH","HTHT","THTHT"]


# Generate initial qubit states in the x-z plane equally spaced.
angles=np.linspace(-np.pi,np.pi,nSampels)
initRotationAxis=[0,1,0]
InitState=np.zeros((len(angles),2),dtype=complex)
for i in range(0,len(angles)):
    InitState[i]=rotation(initRotationAxis,angles[i])@np.array([1,0])

# Apply gates defined in OpList to the inital states.
NewStates=np.zeros((len(OpList),len(InitState),2),dtype=complex)

for i in range(len(OpList)):
    for j in range(len(InitState)):
        NewStates[i,j]=OpList[i]@InitState[j]


# Define some functions for plotting on the Bloch sphere
def PauliExpList(InitState): #Takes in a list of state vectors and provieds the corresponding list on the Bloch sphere
    pauliExpX=np.zeros(len(InitState),dtype=complex)
    pauliExpY=np.zeros(len(InitState),dtype=complex)
    pauliExpZ=np.zeros(len(InitState),dtype=complex)
    for i in range(len(InitState)):
        pauliExpX[i]=InitState[i]@ X @ InitState[i].conj().T
        pauliExpY[i]=InitState[i]@ Y @ InitState[i].conj().T
        pauliExpZ[i]=InitState[i]@ Z @ InitState[i].conj().T
    return [pauliExpX,pauliExpY,pauliExpZ]

def blochPlotFunction(Operator,States,InitStates): #Function to compare initial state to state after applied operator
    b2=Bloch()
    b2.add_points(PauliExpList(InitStates))
    print("Current Operator: ",OpListText[Operator])
    b2.add_points(PauliExpList(States[Operator]))
    b2.render()

interact(blochPlotFunction, Operator=IntSlider(min=0,max=len(NewStates)-1,step=1),States=fixed(NewStates),InitStates=fixed(InitState));    


interactive(children=(IntSlider(value=0, description='Operator', max=6), Output()), _dom_classes=('widget-inte…